# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse     Shipments  \
0  Jul 28 2022  9:20AM  242947        10  SONSB0001886   
1  Jul 28 2022  9:19AM  242946        12      HH-36129   
2  Jul 28 2022  9:19AM  242946        12      HH-36130   
3  Jul 28 2022  9:19AM  242946        12      HH-36131   
4  Jul 28 2022  9:19AM  242946        12      HH-36132   
5  Jul 28 2022  9:19AM  242946        12      HH-36133   
6  Jul 28 2022  9:19AM  242946        12      HH-36134   
7  Jul 28 2022  9:19AM  242946        12      HH-36135   
8  Jul 28 2022  9:11AM  242942        20       8204222   
9  Jul 28 2022  9:11AM  242945        16   AIC-8334268   

                          Customer ShipmentStatus  
0    Nextsource Biotechnology, LLC       Released  
1                        Hush Hush       Released  
2                        Hush Hush       Released  
3                        Hush Hush       Released  
4                        Hush Hush       Released  
5                        Hush Hush       Released  
6                        Hush Hush       Released  
7                        Hush Hush       Released  
8         ACI Healthcare USA, Inc.       Released  
9  American International Chemical       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
44  242942       Released          1
45  242944       Released          1
46  242945       Released          1
47  242946       Released          7
48  242947       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
242942                NaN        NaN       1.0
242944                NaN        NaN       1.0
242945                NaN        NaN       1.0
242946                NaN        NaN       7.0
242947                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
242846               19.0       10.0       1.0
242851                0.0        0.0       1.0
242855                0.0        0.0       1.0
242856                0.0        0.0       1.0
242857                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
242846                 19         10         1
242851                  0          0         1
242855                  0          0         1
242856                  0          0         1
242857                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               242846         19         10         1
1               242851          0          0         1
2               242855          0          0         1
3               242856          0          0         1
4               242857          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               242846        19        10        1
1               242851                            1
2               242855                            1
3               242856                            1
4               242857                            1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                         Customer
0   Jul 28 2022  9:20AM  242947        10    Nextsource Biotechnology, LLC
1   Jul 28 2022  9:19AM  242946        12                        Hush Hush
8   Jul 28 2022  9:11AM  242942        20         ACI Healthcare USA, Inc.
9   Jul 28 2022  9:11AM  242945        16  American International Chemical
10  Jul 28 2022  9:10AM  242939        20         ACI Healthcare USA, Inc.
14  Jul 28 2022  9:04AM  242944        19          GUSA Granules USA, Inc.
15  Jul 28 2022  8:40AM  242941        10                ISDIN Corporation
72  Jul 28 2022  8:36AM  242934        20                Alumier Labs Inc.
84  Jul 28 2022  8:30AM  242914        10                   Citieffe, Inc.
85  Jul 28 2022  8:30AM  242904        10                ISDIN Corporation

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                         Customer  \
0  Jul 28 2022  9:20AM  242947        10    Nextsource Biotechnology, LLC   
1  Jul 28 2022  9:19AM  242946        12                        Hush Hush   
2  Jul 28 2022  9:11AM  242942        20         ACI Healthcare USA, Inc.   
3  Jul 28 2022  9:11AM  242945        16  American International Chemical   
4  Jul 28 2022  9:10AM  242939        20         ACI Healthcare USA, Inc.   
5  Jul 28 2022  9:04AM  242944        19          GUSA Granules USA, Inc.   
6  Jul 28 2022  8:40AM  242941        10                ISDIN Corporation   
7  Jul 28 2022  8:36AM  242934        20                Alumier Labs Inc.   
8  Jul 28 2022  8:30AM  242914        10                   Citieffe, Inc.   
9  Jul 28 2022  8:30AM  242904        10                ISDIN Corporation   

  Completed Executing Released  
0                            1  
1                            7  
2                            1  
3                            1  
4                   1        3  
5                            1  
6                           57  
7                   3        9  
8                            1  
9                   6        6

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                         Customer  \
0  Jul 28 2022  9:20AM  242947        10    Nextsource Biotechnology, LLC   
1  Jul 28 2022  9:19AM  242946        12                        Hush Hush   
2  Jul 28 2022  9:11AM  242942        20         ACI Healthcare USA, Inc.   
3  Jul 28 2022  9:11AM  242945        16  American International Chemical   
4  Jul 28 2022  9:10AM  242939        20         ACI Healthcare USA, Inc.   
5  Jul 28 2022  9:04AM  242944        19          GUSA Granules USA, Inc.   
6  Jul 28 2022  8:40AM  242941        10                ISDIN Corporation   
7  Jul 28 2022  8:36AM  242934        20                Alumier Labs Inc.   
8  Jul 28 2022  8:30AM  242914        10                   Citieffe, Inc.   
9  Jul 28 2022  8:30AM  242904        10                ISDIN Corporation   

  Released Executing Completed  
0        1                      
1        7                      
2        1                      
3        1                      
4        3         1            
5        1                      
6       57                      
7        9         3            
8        1                      
9        6         6

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Jul 28 2022  9:20AM  242947        10    Nextsource Biotechnology, LLC   
1  Jul 28 2022  9:19AM  242946        12                        Hush Hush   
2  Jul 28 2022  9:11AM  242942        20         ACI Healthcare USA, Inc.   
3  Jul 28 2022  9:11AM  242945        16  American International Chemical   
4  Jul 28 2022  9:10AM  242939        20         ACI Healthcare USA, Inc.   

  Released Executing Completed  
0        1                      
1        7                      
2        1                      
3        1                      
4        3         1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Jul 28 2022  9:20AM  242947        10    Nextsource Biotechnology, LLC   
1  Jul 28 2022  9:19AM  242946        12                        Hush Hush   
2  Jul 28 2022  9:11AM  242942        20         ACI Healthcare USA, Inc.   
3  Jul 28 2022  9:11AM  242945        16  American International Chemical   
4  Jul 28 2022  9:10AM  242939        20         ACI Healthcare USA, Inc.   

   Released  Executing  Completed  
0       1.0        NaN        NaN  
1       7.0        NaN        NaN  
2       1.0        NaN        NaN  
3       1.0        NaN        NaN  
4       3.0        1.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Jul 28 2022  9:20AM  242947        10    Nextsource Biotechnology, LLC   
1  Jul 28 2022  9:19AM  242946        12                        Hush Hush   
2  Jul 28 2022  9:11AM  242942        20         ACI Healthcare USA, Inc.   
3  Jul 28 2022  9:11AM  242945        16  American International Chemical   
4  Jul 28 2022  9:10AM  242939        20         ACI Healthcare USA, Inc.   

   Released  Executing  Completed  
0       1.0        0.0        0.0  
1       7.0        0.0        0.0  
2       1.0        0.0        0.0  
3       1.0        0.0        0.0  
4       3.0        1.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3157819     146.0       23.0       25.0
12          242946       7.0        0.0        0.0
16          485852       2.0        0.0        0.0
19         1457471      50.0       12.0        2.0
20         1700353      38.0       16.0        0.0
21          971426       4.0        0.0        0.0
22          242851       1.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3157819     146.0       23.0       25.0
1        12   242946       7.0        0.0        0.0
2        16   485852       2.0        0.0        0.0
3        19  1457471      50.0       12.0        2.0
4        20  1700353      38.0       16.0        0.0
5        21   971426       4.0        0.0        0.0
6        22   242851       1.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     146.0       23.0       25.0
1        12       7.0        0.0        0.0
2        16       2.0        0.0        0.0
3        19      50.0       12.0        2.0
4        20      38.0       16.0        0.0
5        21       4.0        0.0        0.0
6        22       1.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  146.0
1        12  Released    7.0
2        16  Released    2.0
3        19  Released   50.0
4        20  Released   38.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10   12   16    19    20   21   22
Status                                          
Completed   25.0  0.0  0.0   2.0   0.0  0.0  0.0
Executing   23.0  0.0  0.0  12.0  16.0  0.0  0.0
Released   146.0  7.0  2.0  50.0  38.0  4.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10   12   16    19    20   21   22
0          Completed   25.0  0.0  0.0   2.0   0.0  0.0  0.0
1          Executing   23.0  0.0  0.0  12.0  16.0  0.0  0.0
2           Released  146.0  7.0  2.0  50.0  38.0  4.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10   12   16    19    20   21   22
0  Completed   25.0  0.0  0.0   2.0   0.0  0.0  0.0
1  Executing   23.0  0.0  0.0  12.0  16.0  0.0  0.0
2   Released  146.0  7.0  2.0  50.0  38.0  4.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()